Retrieves your historical traded data (positions history) for a specified date range

Formats it as a DataFrame

Saves it to a CSV file

Ideal for trade performance analysis, backtesting, win/loss ratio, equity curve etc.

📦 Function: Get Historical Trade Data and Save to CSV

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import os

def export_trade_history_to_csv(start_date: str, end_date: str, filename: str = "trade_history.csv"):
    """
    Fetch and export historical trade data from MetaTrader5 to a CSV file.

    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        filename (str): File name or path to save trade history (default: 'trade_history.csv').

    Returns:
        bool: True if successful, False otherwise.
    """

    # Connect to MetaTrader 5
    if not mt5.initialize():
        print("MT5 initialization failed:", mt5.last_error())
        return False

    try:
        # Convert string dates to datetime objects
        from_date = datetime.strptime(start_date, "%Y-%m-%d")
        to_date = datetime.strptime(end_date, "%Y-%m-%d")

        # Request history deals
        deals = mt5.history_deals_get(from_date, to_date)

        if deals is None or len(deals) == 0:
            print("No historical trade data found in the given period.")
            mt5.shutdown()
            return False

        # Convert to DataFrame
        deals_df = pd.DataFrame(deals, columns=deals[0]._asdict().keys())
        deals_df['time'] = pd.to_datetime(deals_df['time'], unit='s')

        # Optional sort
        deals_df.sort_values('time', inplace=True)

        # Ensure output directory exists
        os.makedirs(os.path.dirname(filename) or ".", exist_ok=True)

        # Save to CSV
        deals_df.to_csv(filename, index=False)
        print(f"✅ Trade history exported to {filename}")

    except Exception as e:
        print(f"❌ Error during trade export: {e}")
        mt5.shutdown()
        return False

    # Disconnect from MT5
    mt5.shutdown()
    return True
